# Main Project 2 - U-Net을 이용한 이미지 세그멘테이션

In [18]:
import os
import random, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers, utils, Input, Model

import warnings 
warnings.filterwarnings(action='ignore')

### (1) 실습환경 준비

In [ ]:
%pip install datasets
%pip install huggingface_hub

In [2]:
import datasets
from datasets import load_dataset

train_dataset, val_dataset = load_dataset("nateraw/pascal-voc-2012", split=['train', 'val'])
dataset = pd.DataFrame(train_dataset)
dataset

,image,mask
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
...,...,...
1459,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1460,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1461,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1462,<PIL.JpegImagePlugin.JpegImageFile image mode=...,<PIL.PngImagePlugin.PngImageFile image mode=RG...


### (2) 데이터 전처리

In [7]:
from PIL import Image

dataset['reimage'] = None
dataset['remask'] = None

# (572, 572)로 크기 조절
for i in range(len(dataset)):
    dataset['reimage'][i] = np.array(dataset['image'][i].resize((512, 512)))
    dataset['remask'][i] = np.array(dataset['mask'][i].resize((512, 512)))

# 결측치 확인
dataset.isnull().sum()

image      0
mask       0
reimage    0
remask     0
dtype: int64

In [8]:
# data check
print(np.shape(dataset['reimage'][0]))
print(np.shape(dataset['remask'][0]))

(512, 512, 3)
(512, 512, 3)


In [9]:
# train, test set split
train_size = math.floor(len(dataset) * 0.7)
val_size = math.floor(len(dataset) * 0.9)

train_image = dataset['reimage'][:train_size]
train_label = dataset['remask'][:train_size]

val_image = dataset['reimage'][train_size:val_size]
val_label = dataset['remask'][train_size:val_size]

test_image = dataset['reimage'][val_size:]
test_label = dataset['remask'][val_size:]

### (3) U-Net 구현

- Convolution block

In [22]:
def conv_block(inputs, n_filters):
    x = layers.Conv2D(n_filters, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(n_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    return x

- Encoder block

In [23]:
def encode_block(inputs, n_filters):
    x = conv_block(inputs, n_filters)
    e = layers.MaxPooling2D((2, 2))(x)

    return x, e

- Decoder block

In [24]:
def decode_block(inputs, skip, n_filters):
    d = layers.Conv2DTranspose(n_filters, (2, 2), strides=2, padding='same')(inputs)
    x = layers.add([d, skip])
    x = conv_block(x, n_filters)

    return x

- U-Net

In [27]:
def UNET(n_classes):
    inputs = Input(shape=(512, 512, 3))
    
    e1, x = encode_block(inputs, 64)
    e2, x = encode_block(x, 128)
    e3, x = encode_block(x, 256)
    e4, x = encode_block(x, 512)

    x = conv_block(x, 1024)

    x = decode_block(x, e4, 512)
    x = decode_block(x, e3, 256)
    x = decode_block(x, e2, 128)
    x = decode_block(x, e1, 64)

    if n_classes == 1:
        x = layers.Conv2D(n_classes, 1, padding='same', activation='sigmoid')(x)
    else:
        x = layers.Conv2D(n_classes, 1, padding='same', activation='softmax')(x)

    return Model(inputs, x, name='U-Net')

In [32]:
with tf.device('/GPU:0'):
    model = UNET(3)

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=METRICS
    )
    
    model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_98 (Conv2D)          (None, 512, 512, 64)         1792      ['input_7[0][0]']             
                                                                                                  
 batch_normalization_92 (Ba  (None, 512, 512, 64)         256       ['conv2d_98[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_92 (Activation)  (None, 512, 512, 64)         0         ['batch_normalization_92[0

In [34]:
utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
